# Capstone Project - IBM Data Science by Coursera

---

## Week 3, Part 1

#### 1a. Installations

In [358]:
%%capture

%pip install --upgrade pip
%pip install bs4
%pip install pandas

In [359]:
print("Required libraries have been installed.")

Required libraries have been installed.


#### 1b. Imports

In [279]:
# Handeling requests
import requests

# Parsing HTML and XML documents
from bs4 import BeautifulSoup

# Dataframe manipulation
import pandas as pd

print("Required libraries have been imported.")

Required libraries have been imported.


#### 1c. Extract data, and convert to dataframe

In [304]:
# Send 'get' request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# Parse HTML data into BeautifulSoup object
soup = BeautifulSoup(data, 'html.parser')

# Create 3 empty lists, ready for storing table data
postcodes      = []
boroughs       = []
neighbourhoods = []

# Locate table
soup.find('table').find_all('tr')

# Locate all table rows
soup.find('table').find_all('tr')

# For each row of table, find all table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    
# Append data to corresponding lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postcodes.append(cells[0].text.rstrip('\n'))
        boroughs.append(cells[1].text.rstrip('\n'))
        neighbourhoods.append(cells[2].text.rstrip('\n'))
        
# Create new DataFrame from the 3 lists
toronto_df = pd.DataFrame({"Postcode"      : postcodes,
                           "Borough"       : boroughs,
                           "Neighbourhood" : neighbourhoods
                          })
toronto_df.head()

Postcode           Borough              Neighbourhood
0      M1A      Not assigned               Not assigned
1      M2A      Not assigned               Not assigned
2      M3A        North York                  Parkwoods
3      M4A        North York           Victoria Village
4      M5A  Downtown Toronto  Regent Park, Harbourfront

#### 1d. Remove any rows where borough is 'Not assigned'

In [261]:
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

Postcode           Borough                                Neighbourhood
0      M3A        North York                                    Parkwoods
1      M4A        North York                             Victoria Village
2      M5A  Downtown Toronto                    Regent Park, Harbourfront
3      M6A        North York             Lawrence Manor, Lawrence Heights
4      M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

#### 1e. Group neighborhoods by borough

In [375]:
toronto_grouped_df = toronto_df_dropna.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda x: ", ".join(x))
toronto_grouped_df.head()

Postcode      Borough                           Neighbourhood
0      M1B  Scarborough                          Malvern, Rouge
1      M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2      M1E  Scarborough       Guildwood, Morningside, West Hill
3      M1G  Scarborough                                  Woburn
4      M1H  Scarborough                               Cedarbrae

#### 1f. Replace any 'Not assigned' neighborhood values with corresponding borough values

In [376]:
toronto_grouped_df.loc[toronto_df_grouped['Neighbourhood'] == "Not assigned", 'Neighbourhood'] = toronto_df_grouped['Borough']

#### 1g. Create required dataframe

In [377]:
column_names = ['Postcode', 'Borough', 'Neighbourhood']

required_df = pd.DataFrame(columns=column_names)

required_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    required_df = required_df.append(toronto_grouped_df[toronto_grouped_df['Postcode'] == postcode], ignore_index=True)
    
required_df

Postcode           Borough  \
0       M5G  Downtown Toronto   
1       M2H        North York   
2       M4B         East York   
3       M1J       Scarborough   
4       M4G         East York   
5       M4M      East Toronto   
6       M1R       Scarborough   
7       M9V         Etobicoke   
8       M9L        North York   
9       M5V  Downtown Toronto   
10      M1B       Scarborough   
11      M5A  Downtown Toronto   

                                        Neighbourhood  
0                                  Central Bay Street  
1                                   Hillcrest Village  
2                     Parkview Hill, Woodbine Gardens  
3                                 Scarborough Village  
4                                             Leaside  
5                                     Studio District  
6                                   Wexford, Maryvale  
7   South Steeles, Silverstone, Humbergate, Jamest...  
8                                       Humber Summit  
9   CN Tower, King and Spadina, Railway Lands, Har...  
10                                     Malvern, Rouge  
11                          Regent Park, Harbourfront

#### 1h. Print dataframe Shape

In [378]:
toronto_grouped_df.shape

(103, 3)

---

## Week 3, Part 2

#### 2a. Load geospatial coordinates

In [267]:
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
coordinates_df.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

#### 2b. Rename 'Postal Code' column to match main dataframe

In [275]:
coordinates_df.rename(columns={'Postal Code' : 'Postcode'}, inplace=True)
coordinates_df.head()

Postcode   Latitude  Longitude
0      M1B  43.806686 -79.194353
1      M1C  43.784535 -79.160497
2      M1E  43.763573 -79.188711
3      M1G  43.770992 -79.216917
4      M1H  43.773136 -79.239476

#### 2c. Merge coordinates dataframe with main dataframe via 'Postcode' column

In [380]:
toronto_merged_df = toronto_grouped_df.merge(coordinates_df, on='Postcode', how='left')

required_merged_df = required_df.merge(coordinates_df, on='Postcode', how='left')
required_merged_df.head(12)

Postcode           Borough  \
0       M5G  Downtown Toronto   
1       M2H        North York   
2       M4B         East York   
3       M1J       Scarborough   
4       M4G         East York   
5       M4M      East Toronto   
6       M1R       Scarborough   
7       M9V         Etobicoke   
8       M9L        North York   
9       M5V  Downtown Toronto   
10      M1B       Scarborough   
11      M5A  Downtown Toronto   

                                        Neighbourhood   Latitude  Longitude  
0                                  Central Bay Street  43.657952 -79.387383  
1                                   Hillcrest Village  43.803762 -79.363452  
2                     Parkview Hill, Woodbine Gardens  43.706397 -79.309937  
3                                 Scarborough Village  43.744734 -79.239476  
4                                             Leaside  43.709060 -79.363452  
5                                     Studio District  43.659526 -79.340923  
6                                   Wexford, Maryvale  43.750072 -79.295849  
7   South Steeles, Silverstone, Humbergate, Jamest...  43.739416 -79.588437  
8                                       Humber Summit  43.756303 -79.565963  
9   CN Tower, King and Spadina, Railway Lands, Har...  43.628947 -79.394420  
10                                     Malvern, Rouge  43.806686 -79.194353  
11                          Regent Park, Harbourfront  43.654260 -79.360636

---

## Week 3, Part 3

#### 3a. Installations

In [547]:
%%capture

%pip install geopy
%pip install folium
%pip install sklearn
%pip install numpy
%pip install matplotlib

In [548]:
print("Required libraries have been installed.")

Required libraries have been installed.


#### 3b. Imports

In [551]:
# Convert addresses into latitude & longitude values
from geopy.geocoders import Nominatim

# Rendering maps
import folium

# Clustering
from sklearn.cluster import KMeans

# Math
import numpy as np

# Plotting
import matplotlib.cm as cm
import matplotlib.colors as colors

print("Required libraries have been imported.")

Required libraries have been imported.


#### 3c. Identify latitude & longitude values of Toronto

In [402]:
address = 'Toronto'

geolocator = Nominatim(user_agent="can_explorer")
location   = geolocator.geocode(address)

latitude  = location.latitude
longitude = location.longitude

print('Geograpical coordinates of Toronto: ({}, {})'.format(latitude, longitude))

Geograpical coordinates of Toronto: (43.6534817, -79.3839347)


#### 3d. Filter for boroughs that contain the word Toronto, and create new dataframe

In [403]:
borough_names = list(toronto_merged_df['Borough'].unique())

toronto_found = []

for i in borough_names:
    if "toronto" in i.lower():
        toronto_found.append(i)

toronto_filtered_df = toronto_merged_df[toronto_merged_df['Borough'].isin(toronto_found)].reset_index(drop=True)
toronto_filtered_df.head()

Postcode          Borough                   Neighbourhood   Latitude  \
0      M4E     East Toronto                     The Beaches  43.676357   
1      M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2      M4L     East Toronto  India Bazaar, The Beaches West  43.668999   
3      M4M     East Toronto                 Studio District  43.659526   
4      M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  
0 -79.293031  
1 -79.352188  
2 -79.315572  
3 -79.340923  
4 -79.388790

#### 3e. Create map of Toronto to visualize neighbourhoods

In [404]:
fig = folium.Figure(width=900, height=300)

map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(toronto_merged_df['Latitude'], 
                                            toronto_merged_df['Longitude'], 
                                            toronto_merged_df['Borough'], 
                                            toronto_merged_df['Neighbourhood'],
                                           ):
    
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        size=(5,5),
                       ).add_to(map)
    
toronto_map = fig.add_child(map)
toronto_map

#### 3f. Define Foursquare credentials and version

In [405]:
CLIENT_ID     = 'BCFGDA4NNPKQUE15CQ20UDIBLGVBPI3YSTQSCZQLUEMIHYUH'
CLIENT_SECRET = 'QQNSQNVQFXYE0AVDI1DEMZ0M1PRUTJNQZGYHYFLHQCFJMJD4'
VERSION       = '20180605'

#### 3g. Explore top 100 venues within 500 meter radius.

In [535]:
radius = 500
LIMIT  = 100
venues = []

for lat, lng, postcode, borough, neighbourhood in zip(toronto_filtered_df['Latitude'], 
                                                      toronto_filtered_df['Longitude'], 
                                                      toronto_filtered_df['Postcode'], 
                                                      toronto_filtered_df['Borough'], 
                                                      toronto_filtered_df['Neighbourhood'],
                                                      ):
    
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}"\
    .format(CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius, 
            LIMIT,
           )
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((postcode, 
                       borough,
                       neighbourhood,
                       lat, 
                       lng, 
                       venue['venue']['name'], 
                       venue['venue']['location']['lat'], 
                       venue['venue']['location']['lng'],  
                       venue['venue']['categories'][0]['name']
                      ))
        
venues_df = pd.DataFrame(venues)

venues_df.columns = ['Postcode', 
                     'Borough', 
                     'Neighbourhood', 
                     'Borough_Latitude', 
                     'Borough_Longitude', 
                     'Venue_Name', 
                     'Venue_Latitude', 
                     'Venue_Longitude', 
                     'Venue_Category'
                    ]

venues_df.groupby(['Postcode']).count()

venues_df['Venue_Category'].unique()

venues_df.head()

Postcode       Borough                 Neighbourhood  Borough_Latitude  \
0      M4E  East Toronto                   The Beaches         43.676357   
1      M4E  East Toronto                   The Beaches         43.676357   
2      M4E  East Toronto                   The Beaches         43.676357   
3      M4E  East Toronto                   The Beaches         43.676357   
4      M4K  East Toronto  The Danforth West, Riverdale         43.679557   

   Borough_Longitude                          Venue_Name  Venue_Latitude  \
0         -79.293031                   Glen Manor Ravine       43.676821   
1         -79.293031  The Big Carrot Natural Food Market       43.678879   
2         -79.293031                 Grover Pub and Grub       43.679181   
3         -79.293031                       Upper Beaches       43.680563   
4         -79.352188                       MenEssentials       43.677820   

   Venue_Longitude     Venue_Category  
0       -79.293942              Trail  
1       -79.297734  Health Food Store  
2       -79.297215                Pub  
3       -79.292869       Neighborhood  
4       -79.351265     Cosmetics Shop

#### 3h. One hot encoding

In [536]:
toronto_onehot_df = pd.get_dummies(venues_df[['Venue_Category']], prefix="", prefix_sep="")

# Add neighbourhood column
toronto_onehot_df['Neighbourhood'] = venues_df['Neighbourhood'] 

# Shift neighbourhood column to first column
fixed_columns     = list(toronto_onehot_df.columns[-1:]) + list(toronto_onehot_df.columns[:-3])
toronto_onehot_df = toronto_onehot_df[fixed_columns]

print(toronto_onehot_df.shape)
toronto_onehot_df.head()

(1638, 234)


Neighbourhood  Afghan Restaurant  Airport  \
0                   The Beaches                  0        0   
1                   The Beaches                  0        0   
2                   The Beaches                  0        0   
3                   The Beaches                  0        0   
4  The Danforth West, Riverdale                  0        0   

   Airport Food Court  Airport Lounge  Airport Service  Airport Terminal  \
0                   0               0                0                 0   
1                   0               0                0                 0   
2                   0               0                0                 0   
3                   0               0                0                 0   
4                   0               0                0                 0   

   American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0                    0             0         0            0           0   
1                    0             0         0            0           0   
2                    0             0         0            0           0   
3                    0             0         0            0           0   
4                    0             0         0            0           0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Workshop  \
0                    0                 0                   0              0   
1                    0                 0                   0              0   
2                    0                 0                   0              0   
3                    0                 0                   0              0   
4                    0                 0                   0              0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  Baseball Stadium  \
0          0           0           0       0     0    0                 0   
1          0           0           0       0     0    0                 0   
2          0           0           0       0     0    0                 0   
3          0           0           0       0     0    0                 0   
4          0           0           0       0     0    0                 0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                   0      0                0         0           0   
1                   0      0                0         0           0   
2                   0      0                0         0           0   
3                   0      0                0         0           0   
4                   0      0                0         0           0   

   Belgian Restaurant  Bistro  Boat or Ferry  Bookstore  Boutique  \
0                   0       0              0          0         0   
1                   0       0              0          0         0   
2                   0       0              0          0         0   
3                   0       0              0          0         0   
4                   0       0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bubble Tea Shop  Building  \
0                     0               0        0                0         0   
1                     0               0        0                0         0   
2                     0               0        0                0         0   
3                     0               0        0                0         0   
4                     0               0        0                0         0   

   Burger Joint  Burrito Place  Bus Line  Business Service  Butcher  \
0             0              0         0                 0        0   
1             0              0         0                 0        0   
2             0              0         0                 0        0   
3             0              0         0                 0        0   
4             0              0         0                 0        0   

   Cafeteria  Café  Cajun / Creole Restaurant  Ca

#### 3i. Calculate mean frequency of occurrence for each category, and group rows by neighbourhood

In [537]:
toronto_onehot_grouped_df = toronto_onehot_df.groupby(['Neighbourhood']).mean().reset_index()

print(toronto_onehot_grouped_df.shape)
toronto_onehot_grouped_df.head()

(39, 234)


Neighbourhood  Afghan Restaurant  \
0                                        Berczy Park                0.0   
1       Brockton, Parkdale Village, Exhibition Place                0.0   
2  Business reply mail Processing Centre, South C...                0.0   
3  CN Tower, King and Spadina, Railway Lands, Har...                0.0   
4                                 Central Bay Street                0.0   

    Airport  Airport Food Court  Airport Lounge  Airport Service  \
0  0.000000            0.000000        0.000000         0.000000   
1  0.000000            0.000000        0.000000         0.000000   
2  0.000000            0.000000        0.000000         0.000000   
3  0.058824            0.058824        0.117647         0.176471   
4  0.000000            0.000000        0.000000         0.000000   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  Art Gallery  \
0          0.000000                  0.0           0.0       0.0     0.017241   
1          0.000000                  0.0           0.0       0.0     0.000000   
2          0.000000                  0.0           0.0       0.0     0.000000   
3          0.117647                  0.0           0.0       0.0     0.000000   
4          0.000000                  0.0           0.0       0.0     0.000000   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0    0.000000                  0.0               0.0                 0.0   
1    0.000000                  0.0               0.0                 0.0   
2    0.000000                  0.0               0.0                 0.0   
3    0.000000                  0.0               0.0                 0.0   
4    0.015385                  0.0               0.0                 0.0   

   Auto Workshop  BBQ Joint  Baby Store  Bagel Shop    Bakery  Bank       Bar  \
0         0.0000   0.017241         0.0    0.017241  0.034483   0.0  0.000000   
1         0.0000   0.000000         0.0    0.000000  0.045455   0.0  0.045455   
2         0.0625   0.000000         0.0    0.000000  0.000000   0.0  0.000000   
3         0.0000   0.000000         0.0    0.000000  0.000000   0.0  0.058824   
4         0.0000   0.000000         0.0    0.000000  0.000000   0.0  0.015385   

   Baseball Stadium  Basketball Stadium     Beach  Bed & Breakfast  Beer Bar  \
0               0.0            0.017241  0.017241              0.0  0.034483   
1               0.0            0.000000  0.000000              0.0  0.000000   
2               0.0            0.000000  0.000000              0.0  0.000000   
3               0.0            0.000000  0.000000              0.0  0.000000   
4               0.0            0.000000  0.000000              0.0  0.000000   

   Beer Store  Belgian Restaurant    Bistro  Boat or Ferry  Bookstore  \
0         0.0                 0.0  0.017241       0.000000        0.0   
1         0.0                 0.0  0.000000       0.000000        0.0   
2         0.0                 0.0  0.000000       0.000000        0.0   
3         0.0                 0.0  0.000000       0.058824        0.0   
4         0.0                 0.0  0.000000       0.000000        0.0   

   Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  Bubble Tea Shop  \
0  0.000000                   0.0        0.017241   0.0000         0.000000   
1  0.000000                   0.0        0.090909   0.0000         0.000000   
2  0.000000                   0.0        0.000000   0.0625         0.000000   
3  0.058824                   0.0        0.000000   0.0000         0.000000   
4  0.000000                   0.0        0.000000   0.0000         0.030769   

   Building  Burger Joint  Burrito Place  Bus Line  Business Service  Butcher  \
0       0.0      0.000000       0.000000       0.0          0.000000   0.0000   
1       0.0      0.000000       0.045455       0.0          0.000000   0.0000   
2       0.0      0.000000       0.062500       0.0          0.000000   0.0625   
3       0.0      0.000000    

#### 3h. Define a function to sort venues into decending order

In [538]:
def sortTopVenues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### 3i. Explore top 10 venues within each neighbourhood

In [539]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted_df = pd.DataFrame(columns=columns)
venues_sorted_df['Neighbourhood'] = toronto_onehot_grouped_df['Neighbourhood']

for ind in np.arange(toronto_onehot_grouped_df.shape[0]):
    venues_sorted_df.iloc[ind, 1:] = sortTopVenues(toronto_onehot_grouped_df.iloc[ind, :], num_top_venues)

venues_sorted_df.head()

Neighbourhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...    Light Rail Station   
3  CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                  Café           Cheese Shop              Beer Bar   
1           Coffee Shop        Breakfast Spot                Bakery   
2      Recording Studio  Fast Food Restaurant        Farmers Market   
3        Airport Lounge      Airport Terminal           Coffee Shop   
4        Sandwich Place   Japanese Restaurant    Italian Restaurant   

  5th Most Common Venue  6th Most Common Venue 7th Most Common Venue  \
0                Bakery     Seafood Restaurant            Restaurant   
1     Convenience Store  Performing Arts Venue             Pet Store   
2               Brewery             Restaurant         Auto Workshop   
3       Harbor / Marina                  Plane   Rental Car Location   
4                  Café       Department Store          Burger Joint   

  8th Most Common Venue 9th Most Common Venue      10th Most Common Venue  
0        Farmers Market          Cocktail Bar           Indian Restaurant  
1          Climbing Gym            Restaurant               Burrito Place  
2               Butcher         Burrito Place                 Pizza Place  
3      Sculpture Garden         Boat or Ferry                         Bar  
4       Bubble Tea Shop           Salad Place  Modern European Restaurant

#### 3j. Use k-means to create 5 clusters of Toronto areas.

In [545]:
k_clusters = 5

# Drop columns such that only latitude & longitude remain
toronto_cluster_df = toronto_onehot_grouped_df.drop(['Neighbourhood'], axis=1)

# Run k-means clustering
k_means = KMeans(n_clusters=k_clusters, random_state=0).fit(toronto_cluster_df)

# Create new dataframe to include cluster & top 10 venues
toronto_cluster_top_venues_df = toronto_filtered_df.copy()

# Add clustering labels
toronto_cluster_top_venues_df["Cluster Labels"] = k_means.labels_

# Merge dataframes to add latitude & longitude columns
toronto_cluster_top_venues_df = toronto_cluster_top_venues_df.join(venues_sorted_df.set_index('Neighbourhood'), on='Neighbourhood')

# Sort by cluster labels
toronto_cluster_top_venues_df.sort_values(["Cluster Labels"], inplace=True)

print(toronto_cluster_top_venues_df.shape)
toronto_cluster_top_venues_df.tail()


(39, 16)


Postcode           Borough  \
20      M5K  Downtown Toronto   
26      M5T  Downtown Toronto   
12      M4Y  Downtown Toronto   
18      M5H  Downtown Toronto   
27      M5V  Downtown Toronto   

                                        Neighbourhood   Latitude  Longitude  \
20           Toronto Dominion Centre, Design Exchange  43.647177 -79.381576   
26          Kensington Market, Chinatown, Grange Park  43.653206 -79.400049   
12                               Church and Wellesley  43.665860 -79.383160   
18                           Richmond, Adelaide, King  43.650571 -79.384568   
27  CN Tower, King and Spadina, Railway Lands, Har...  43.628947 -79.394420   

    Cluster Labels 1st Most Common Venue          2nd Most Common Venue  \
20               1           Coffee Shop                          Hotel   
26               2                  Café  Vegetarian / Vegan Restaurant   
12               2           Coffee Shop            Japanese Restaurant   
18               3           Coffee Shop                           Café   
27               4       Airport Service                 Airport Lounge   

   3rd Most Common Venue  4th Most Common Venue 5th Most Common Venue  \
20                  Café             Restaurant    Seafood Restaurant   
26           Coffee Shop  Vietnamese Restaurant                   Bar   
12      Sushi Restaurant             Restaurant               Gay Bar   
18        Clothing Store             Restaurant                   Gym   
27      Airport Terminal            Coffee Shop       Harbor / Marina   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
20           Salad Place   American Restaurant   Japanese Restaurant   
26    Mexican Restaurant          Dessert Shop          Burger Joint   
12       Bubble Tea Shop                  Café                   Pub   
18                 Hotel            Steakhouse       Thai Restaurant   
27                 Plane   Rental Car Location      Sculpture Garden   

   9th Most Common Venue 10th Most Common Venue  
20      Asian Restaurant    Sporting Goods Shop  
26                  Park                 Bakery  
12           Men's Store                  Hotel  
18        Cosmetics Shop                    Bar  
27         Boat or Ferry                    Bar

#### 3k. Visualise clusters

In [555]:
fig2 = folium.Figure(width=900, height=300)

map2 = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lng, post, bor, hood, cluster in zip(toronto_cluster_top_venues_df['Latitude'], 
                                              toronto_cluster_top_venues_df['Longitude'], 
                                              toronto_cluster_top_venues_df['Postcode'], 
                                              toronto_cluster_top_venues_df['Borough'], 
                                              toronto_cluster_top_venues_df['Neighbourhood'], 
                                              toronto_cluster_top_venues_df['Cluster Labels'],
                                             ):
    
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, hood, cluster), parse_html=True)
    
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color=rainbow[cluster-1],
                        fill=True,
                        fill_color=rainbow[cluster-1],
                        fill_opacity=0.7,
                       ).add_to(map2)
       
map_clusters = fig.add_child(map2)
map_clusters